In [0]:
#Install dependencies
!pip install syft
!pip install torch

In [4]:
# import dependencies
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import syft as sy


W0820 14:56:16.005809 139905971980160 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0820 14:56:16.025116 139905971980160 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
# make workers
hook = sy.TorchHook(torch)
bob =  sy.VirtualWorker(hook, id="bob")
alice =  sy.VirtualWorker(hook, id="alice")

In [0]:
# define the arguments used in the model
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = 10
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 10
        self.save_model = False

args = Arguments()


In [7]:
torch.manual_seed(args.seed)

In [0]:
use_cuda = not args.no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [0]:
federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
    .federate((bob, alice)), 
    batch_size=args.batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [0]:

model = Net()
model = model.to(device)  #pushing the model into available device.
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [12]:

for epoch in range(1, args.epochs + 1):
    # Train the model
    model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader): # iterate through each worker's dataset
        
        model.send(data.location) #send the model to the right location ; data.location returns the worker name in which the data is present
        
        data, target = data.to(device), target.to(device) # pushing both the data and target labels onto the available device.
        
        optimizer.zero_grad() # 1) erase previous gradients (if they exist)
        output = model(data)  # 2) make a prediction
        loss = F.nll_loss(output, target)  # 3) calculate how much we missed
        loss.backward()  # 4) figure out which weights caused us to miss
        optimizer.step()  # 5) change those weights
        model.get()  # get the model back (with gradients)
        
        if batch_idx % args.log_interval == 0:
            loss = loss.get() #get the loss back
            print('Epoch: {} [Training: {:.0f}%]\tLoss: {:.6f}'.format(epoch, 100. * batch_idx / len(federated_train_loader), loss.item()))
    # Test the model
    model.eval()  
    test_loss = 0
    correct = 0
    with torch.no_grad():
      for data, target in test_loader:
        data, target = data.to(device), target.to(device) 
        output = model(data) # Getting a prediction

        test_loss += F.nll_loss(output, target, reduction='sum').item() #updating test loss
        pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 

        correct += pred.eq(target.view_as(pred)).sum().item() #correct pred in the current test set.

    test_loss /= len(test_loader.dataset) 

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))

    torch.save(model.state_dict(), "mnist_cnn.pt")

    
    

Epoch: 1 [Training: 0%]	Loss: 2.305134
Epoch: 1 [Training: 1%]	Loss: 2.273475
Epoch: 1 [Training: 2%]	Loss: 2.216174
Epoch: 1 [Training: 3%]	Loss: 2.156802
Epoch: 1 [Training: 4%]	Loss: 2.139429
Epoch: 1 [Training: 5%]	Loss: 2.053059
Epoch: 1 [Training: 6%]	Loss: 1.896587
Epoch: 1 [Training: 7%]	Loss: 1.917239
Epoch: 1 [Training: 9%]	Loss: 1.655076
Epoch: 1 [Training: 10%]	Loss: 1.440329
Epoch: 1 [Training: 11%]	Loss: 1.231347
Epoch: 1 [Training: 12%]	Loss: 0.983715
Epoch: 1 [Training: 13%]	Loss: 0.867023
Epoch: 1 [Training: 14%]	Loss: 0.890953
Epoch: 1 [Training: 15%]	Loss: 0.861902
Epoch: 1 [Training: 16%]	Loss: 0.654317
Epoch: 1 [Training: 17%]	Loss: 0.587091
Epoch: 1 [Training: 18%]	Loss: 0.693011
Epoch: 1 [Training: 19%]	Loss: 0.593099
Epoch: 1 [Training: 20%]	Loss: 0.531986
Epoch: 1 [Training: 21%]	Loss: 0.400264
Epoch: 1 [Training: 22%]	Loss: 0.455322
Epoch: 1 [Training: 23%]	Loss: 0.438987
Epoch: 1 [Training: 25%]	Loss: 0.398361
Epoch: 1 [Training: 26%]	Loss: 0.371324
Epoch: 1 